# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [143]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [148]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [257]:
cur.close()
conn.close()

In [13]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.relpath(f))  # NOTE: This was edited as I was using my own system
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [90]:
song_files = get_files(filepath)

In [133]:
song_files[0]

'data/song_data/A/A/A/TRAAAEF128F4273421.json'

In [7]:
filepath = 'data/song_data'

In [99]:
df = pd.read_json(song_files[0], lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [127]:
song_data = (
    df[['song_id', 'title', 'artist_id',
        'year', 'duration']].values
)
song_data = list(song_data[0])
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [128]:
song_table_insert = ("""
    INSERT INTO songs (song_id, title, artist_id, year, duration)
    VALUES (%s, %s, %s, %s, %s);
""")

cur.execute(song_table_insert, song_data)
conn.commit()

In [129]:
cur.execute("""
    SELECT * FROM songs;
""")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [134]:
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982


In [145]:
artist_data = (
    df[['artist_id', 'artist_name', 'artist_location',
        'artist_latitude', 'artist_longitude']].values
)
artist_data = list(artist_data[0])
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [146]:
artist_table_insert = ("""
    INSERT INTO artists
        (artist_id, name, location, latitude, longitude)
    VALUES (%s, %s, %s, %s, %s);
""")

In [149]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

In [150]:
cur.execute("""
    SELECT * FROM artists;
""")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', Decimal('NaN'), Decimal('NaN'))


# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [156]:
log_files = get_files(filepath)
len(log_files)

30

In [151]:
filepath = 'data/log_data'

In [159]:
df = pd.read_json(log_files[0], lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [160]:
is_NextSong = df['page'] == 'NextSong'
df = df[is_NextSong]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [81]:
# Python version
import datetime
datetime.datetime.fromtimestamp(1541377992796/1000.0)

# (year, month, day, hour, minute, second, microsecond)
# datetime.datetime(2018, 11, 4, 16, 33, 12, 796000)

# Pandas version
pd.to_datetime(1541377992796, unit='ms')
# Timestamp('2018-11-05 00:33:12.796000')
# pd.to_datetime(1541377992796, unit='ms').day_name() - 'Monday'

Timestamp('2018-11-05 00:33:12.796000')

In [246]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [218]:
t = df['ts']
t.head()

0   2018-11-11 02:33:56.796
1   2018-11-11 02:36:10.796
2   2018-11-11 02:40:34.796
4   2018-11-11 04:36:13.796
5   2018-11-11 04:36:46.796
Name: ts, dtype: datetime64[ns]

In [247]:
time_data = list((t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.day_name()))
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'day_of_week')
time_data

[0    2018-11-11 02:33:56.796
 1    2018-11-11 02:36:10.796
 2    2018-11-11 02:40:34.796
 4    2018-11-11 04:36:13.796
 5    2018-11-11 04:36:46.796
 6    2018-11-11 09:56:28.796
 7    2018-11-11 10:00:47.796
 8    2018-11-11 10:04:07.796
 9    2018-11-11 10:08:28.796
 10   2018-11-11 10:12:00.796
 12   2018-11-11 10:27:43.796
 13   2018-11-11 10:33:44.796
 14   2018-11-11 10:36:22.796
 15   2018-11-11 10:40:14.796
 16   2018-11-11 10:43:23.796
 17   2018-11-11 10:46:18.796
 23   2018-11-11 11:14:51.796
 25   2018-11-11 13:41:22.796
 27   2018-11-11 13:52:23.796
 28   2018-11-11 13:58:22.796
 30   2018-11-11 14:05:43.796
 31   2018-11-11 14:07:54.796
 32   2018-11-11 14:09:36.796
 33   2018-11-11 14:14:29.796
 35   2018-11-11 14:15:06.796
 36   2018-11-11 14:17:52.796
 37   2018-11-11 14:19:37.796
 38   2018-11-11 14:24:58.796
 39   2018-11-11 14:30:05.796
 40   2018-11-11 14:33:53.796
                ...          
 57   2018-11-11 15:10:12.796
 58   2018-11-11 15:14:56.796
 59   2018

In [252]:
time_dict = dict(zip(column_labels, time_data))
time_dict

{'timestamp': 0    2018-11-11 02:33:56.796
 1    2018-11-11 02:36:10.796
 2    2018-11-11 02:40:34.796
 4    2018-11-11 04:36:13.796
 5    2018-11-11 04:36:46.796
 6    2018-11-11 09:56:28.796
 7    2018-11-11 10:00:47.796
 8    2018-11-11 10:04:07.796
 9    2018-11-11 10:08:28.796
 10   2018-11-11 10:12:00.796
 12   2018-11-11 10:27:43.796
 13   2018-11-11 10:33:44.796
 14   2018-11-11 10:36:22.796
 15   2018-11-11 10:40:14.796
 16   2018-11-11 10:43:23.796
 17   2018-11-11 10:46:18.796
 23   2018-11-11 11:14:51.796
 25   2018-11-11 13:41:22.796
 27   2018-11-11 13:52:23.796
 28   2018-11-11 13:58:22.796
 30   2018-11-11 14:05:43.796
 31   2018-11-11 14:07:54.796
 32   2018-11-11 14:09:36.796
 33   2018-11-11 14:14:29.796
 35   2018-11-11 14:15:06.796
 36   2018-11-11 14:17:52.796
 37   2018-11-11 14:19:37.796
 38   2018-11-11 14:24:58.796
 39   2018-11-11 14:30:05.796
 40   2018-11-11 14:33:53.796
                ...          
 57   2018-11-11 15:10:12.796
 58   2018-11-11 15:14:56.7

In [256]:
time_df = pd.DataFrame(time_dict)
time_df

,timestamp,hour,day,week,month,year,day_of_week
0,2018-11-11 02:33:56.796,2,11,45,11,2018,Sunday
1,2018-11-11 02:36:10.796,2,11,45,11,2018,Sunday
2,2018-11-11 02:40:34.796,2,11,45,11,2018,Sunday
4,2018-11-11 04:36:13.796,4,11,45,11,2018,Sunday
5,2018-11-11 04:36:46.796,4,11,45,11,2018,Sunday
6,2018-11-11 09:56:28.796,9,11,45,11,2018,Sunday
7,2018-11-11 10:00:47.796,10,11,45,11,2018,Sunday
8,2018-11-11 10:04:07.796,10,11,45,11,2018,Sunday
9,2018-11-11 10:08:28.796,10,11,45,11,2018,Sunday
10,2018-11-11 10:12:00.796,10,11,45,11,2018,Sunday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
time_table_insert = ("""
    INSERT INTO artists ()
""")

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.